In [31]:
import git
from pathlib import Path
import os
import numpy as np
import tqdm
import scipy.ndimage as ndi
import pandas as pd
from skimage.filters import gabor_kernel
from scipy.signal import fftconvolve

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from learned import *

np.random.seed(0)

In [32]:
DATASET = "agriVision"
RAW_DATA_SUFFIX = "batch0-agriVision-RGB-cleaned-jitter 2"
FINAL_DATA_NAME = "agriVision-full"
PARAM_CSV = "gabor_new.csv"
CONSTANT_SAMPLE_SIZE = int(1e5)

num_images=1000
jitter=False 
normalize=False 

#data_dir = os.path.join(ROOT_DIR, 'raw-data', DATASET, RAW_DATA_SUFFIX)
data_dir = os.path.join(ROOT_DIR, 'raw-data', RAW_DATA_SUFFIX)

In [33]:
def _generate_gabor_kernel_skimage_cached(frequency: float,
                                          wave_number: int,
                                          theta: float,
                                          aspect_ratio: float,
                                          dtype_str: str) -> np.ndarray:
    if frequency <= 0:
        raise ValueError("frequency must be positive (cycles/pixel).")
    
    # choose n_stds explicitly (same value you pass to gabor_kernel; default is 3)
    n_stds = 3
    sigma_x = wave_number / (2.0 * n_stds * frequency)   # NOT 4*frequency
    sigma_y = aspect_ratio * sigma_x
    k_complex = gabor_kernel(frequency=frequency, theta=theta, sigma_x=sigma_x, sigma_y=sigma_y, n_stds=n_stds)

    k = np.real(k_complex).astype(np.dtype(dtype_str), copy=False)
    k -= k.mean()
    nrm = np.linalg.norm(k.ravel())
    if nrm > 1e-12:
        k = k / nrm
    return k

def generate_gabor_kernel_skimage(frequency: float,
                                  wave_number: int,
                                  theta: float = 0.0,
                                  aspect_ratio: float = 1.0,
                                  dtype=np.float64) -> np.ndarray:
    """Public wrapper returning a copy to avoid accidental mutation of cached data."""
    arr = _generate_gabor_kernel_skimage_cached(
        float(frequency), int(wave_number), float(theta),
        float(aspect_ratio), np.dtype(dtype).name
    )
    if arr is not None:
        return arr.copy()
    return None

In [34]:
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))
# param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", "gabor_test.csv"))

orientations = [0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6]

filters = []
for i in range(len(param_df)):
    fil_orients = []

    for j in range(len(orientations)):
        fil = generate_gabor_kernel_skimage(param_df["frequency"][i], param_df["wave_number"][i], orientations[j], param_df["aspect_ratio"][i])
        if fil is not None:
            fil_orients.append(fil)
    
    filters.append(fil_orients)

print(filters[0][0].shape)
print(len(filters), len(filters[0]))

(25, 47)
42 6


In [35]:
for i in range(len(filters)):
    for j in range(len(filters[i])):
        print(f"index: {i}, params: {tuple(param_df.loc[i])[1:]}, orientation: {orientations[j]:.3f}, size: {filters[i][j].shape}")

index: 0, params: (2.0, 0.5, 0.044), orientation: 0.000, size: (25, 47)
index: 0, params: (2.0, 0.5, 0.044), orientation: 0.524, size: (25, 41)
index: 0, params: (2.0, 0.5, 0.044), orientation: 1.047, size: (41, 25)
index: 0, params: (2.0, 0.5, 0.044), orientation: 1.571, size: (47, 25)
index: 0, params: (2.0, 0.5, 0.044), orientation: 2.094, size: (41, 25)
index: 0, params: (2.0, 0.5, 0.044), orientation: 2.618, size: (25, 41)
index: 1, params: (2.0, 0.5, 0.065), orientation: 0.000, size: (17, 33)
index: 1, params: (2.0, 0.5, 0.065), orientation: 0.524, size: (17, 29)
index: 1, params: (2.0, 0.5, 0.065), orientation: 1.047, size: (29, 17)
index: 1, params: (2.0, 0.5, 0.065), orientation: 1.571, size: (33, 17)
index: 1, params: (2.0, 0.5, 0.065), orientation: 2.094, size: (29, 17)
index: 1, params: (2.0, 0.5, 0.065), orientation: 2.618, size: (17, 29)
index: 2, params: (2.0, 0.5, 0.096), orientation: 0.000, size: (13, 23)
index: 2, params: (2.0, 0.5, 0.096), orientation: 0.524, size: (

In [36]:
images = load_images_from_directory(data_dir, n=num_images, jitter=jitter, normalize=normalize)
print("num_images, H, W, channels:", images.shape, "\nnum_filters", len(filters))

Loading images:   0%|          | 0/1000 [00:00<?, ?it/s]

num_images, H, W, channels: (1000, 512, 512, 3) 
num_filters 42


In [ ]:

def rgb_to_gray(img): # assuming BGR input from OpenCV
    if img.ndim == 2:
        return np.asarray(img, dtype=np.float32)
    # BGR order from OpenCV
    b, g, r = img[..., 0], img[..., 1], img[..., 2]
    # same luminance weights but mapped to correct channels
    return (0.2989 * r + 0.5870 * g + 0.1140 * b).astype(np.float32)

def get_channel_2d(img, which):
    if which == "gray":
        return rgb_to_gray(img)
    if img.ndim == 2:
        return np.asarray(img, dtype=np.float32)
    # BGR channel map
    ch_map = {"blue": 0, "green": 1, "red": 2}
    return np.asarray(img[..., ch_map[which]], dtype=np.float32)


def apply_filter(image2d, kernel):
    """
    Convolve a single-channel (H,W) image with kernel using FFT.
    Minimal memory; returns float32 (H,W).
    """
    img = np.asarray(image2d, dtype=np.float32, order="C")
    ker = np.asarray(kernel, dtype=np.float32, order="C")
    out = fftconvolve(img, ker, mode="same").astype(np.float32, copy=False)
    return out



  0%|          | 0/42 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:


def transform_by_channel(
    images,
    filters,
    channel_tag,
    CONSTANT_SAMPLE_SIZE,
    images_per_orient=166,
    rng: np.random.Generator | None = None,
):
    """
    Run the Gabor pipeline for one channel (gray/r/g/b) and return results without saving.

    Returns:
        filter_data_map: dict[int, np.ndarray]  # orientation index -> subsampled coefficients (1D)
        total_samples: int                      # total scalar samples if each image were single-channel
        per_filter_len: int                     # length of subsampled vector per orientation
    """
    if rng is None:
        rng = np.random.default_rng()

    filter_data_map = dict()

    # main loop: over orientations (i) and filters within orientation (j)
    for i in tqdm(range(len(filters)), desc=f"Channel={channel_tag}"):
        transformed = []
        for j in range(len(filters[i])):
            curr_transformed = []
            fil = filters[i][j]

            # choose a subset of images for this (i, j)
            indices = rng.choice(len(images), size=images_per_orient, replace=False)
            image_subset = images[indices]

            for img in image_subset:
                ch2d = get_channel_2d(img, channel_tag)   # (H, W)
                convolved = apply_filter(ch2d, fil)        # (H, W)
                curr_transformed.append(convolved.flatten())

            # stack all selected images for this (i,j)
            curr_transformed = np.hstack(curr_transformed)

            # intermediate subsample to limit size
            take = min(curr_transformed.size, CONSTANT_SAMPLE_SIZE)
            idx = np.round(np.linspace(0, curr_transformed.size - 1, take)).astype(int)
            curr_transformed = np.sort(curr_transformed)[idx]

            # accumulate for this orientation i
            transformed = (
                np.concatenate((transformed, curr_transformed))
                if len(transformed) else curr_transformed
            )

        # append the negation of all coefficients to account for other orientations
        neg_transformed = -1.0 * transformed
        transformed = np.concatenate((transformed, neg_transformed))

        # final subsample for this orientation i
        take = min(transformed.size, CONSTANT_SAMPLE_SIZE)
        idx = np.round(np.linspace(0, transformed.size - 1, take)).astype(int)
        transformed = np.sort(transformed)[idx]

        filter_data_map[i] = transformed

    # Compute total scalar samples if each image were single-channel
    N = len(images)
    sample_img = get_channel_2d(images[0], channel_tag)
    total_samples = int(N * np.prod(sample_img.shape))

    # per-filter length (use orientation 0 which must exist if filters are non-empty)
    first_key = next(iter(filter_data_map))
    per_filter_len = len(filter_data_map[first_key])

    return filter_data_map, total_samples, per_filter_len


def save_transform_maps(
    filter_data_map: dict,
    total_samples: int,
    ROOT_DIR: str,
    FINAL_DATA_NAME: str,
    channel_tag: str,
):
    """
    Save the maps/sizes to disk with channel-specific suffixes.
    """
    out_dir = os.path.join(ROOT_DIR, "transformed-data")
    os.makedirs(out_dir, exist_ok=True)

    data_output_file = os.path.join(
        out_dir, f"{FINAL_DATA_NAME}-gabor-{channel_tag}.pickle"
    )
    size_output_file = os.path.join(
        out_dir, f"{FINAL_DATA_NAME}-gabor-{channel_tag}-size.pickle"
    )

    print(f"[{channel_tag}] Without subsampling size:", total_samples)
    # show any one entry length as the per-orientation subsample length
    some_key = next(iter(filter_data_map))
    print(f"[{channel_tag}] With subsampling, per-orientation:", len(filter_data_map[some_key]))

    pd.to_pickle(filter_data_map, data_output_file)
    pd.to_pickle({i: total_samples for i in range(len(filter_data_map))}, size_output_file)


In [ ]:
# Gray
gray_map, gray_total, _ = transform_by_channel(
    images, filters, "gray", CONSTANT_SAMPLE_SIZE, images_per_orient=166
)
save_transform_maps(gray_map, gray_total, ROOT_DIR, FINAL_DATA_NAME, "gray")

# Red
red_map, red_total, _ = transform_by_channel(
    images, filters, "red", CONSTANT_SAMPLE_SIZE, images_per_orient=166
)
save_transform_maps(red_map, red_total, ROOT_DIR, FINAL_DATA_NAME, "red")

# Green
green_map, green_total, _ = transform_by_channel(
    images, filters, "green", CONSTANT_SAMPLE_SIZE, images_per_orient=166
)
save_transform_maps(green_map, green_total, ROOT_DIR, FINAL_DATA_NAME, "green")

# Blue
blue_map, blue_total, _ = transform_by_channel(
    images, filters, "blue", CONSTANT_SAMPLE_SIZE, images_per_orient=166
)
save_transform_maps(blue_map, blue_total, ROOT_DIR, FINAL_DATA_NAME, "blue")
